In [1]:
import tensorflow as tf
import nltk
import re
import pandas as pd
import numpy as np
import os
from pathlib import Path
from scipy import stats

In [2]:
test_eval_data = "/Users/zxj/Desktop/study/semester3/MCS/humor/data/task-1/data/task-1/test_eval.csv"

test_eval=pd.read_csv(test_eval_data)
test_eval_label=test_eval.meanGrade

#get orignial news
test_eval_news=test_eval.original
test_eval_funny_word=test_eval.edit

In [3]:
from nltk.corpus import stopwords
stopwords=set(stopwords.words('english'))
tt=nltk.tokenize.regexp.WordPunctTokenizer()
lemmatizer=nltk.stem.wordnet.WordNetLemmatizer()

punctuation = '!,;:?"\'.\'/<>'
def removePunctuation(text):
    text = re.sub(r'[{}]+'.format(punctuation),'',text)
    return text.strip()

#catch the changed word
def find_tag(news):
    p = re.compile(r'[<](.*?)/[>]', re.S)
    return re.findall(p, news)

def preprocess_news(sentence,funny_word):
    final=[]
    num=0
    bad_sign=["’","‘","-"]
    
    for event in sentence:
        lines=[]       
        event=str(event)
        changed_word=find_tag(event)   
        event=removePunctuation(event)
        event=tt.tokenize(event)
        #tokenize and remove Punctuation
        
        for i in event:
            if i not in bad_sign:
                
                if i in changed_word:
                    #add the funny behind the changed word
                    #lines.append(i)
                    lines.append(funny_word[num])
                else:
                    lines.append(i)
                    
        lines=' '.join(lines)
        lines='[CLS]'+' '+lines+' '+'[SEP]'
        final.append(lines)
        num+=1
    return final

test_eval_news=preprocess_news(test_eval_news,test_eval_funny_word)

In [4]:
from transformers import AlbertTokenizer, TFAlbertModel
#import tensorflow as tf
import numpy as np

def import_model(filename):
    tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2',token_ids_1=None)
    inputs_train = tokenizer(test_eval_news,padding=True,return_tensors="pt")
    inputs_train=np.array(inputs_train['input_ids'])
    input_layer=tf.keras.Input(shape=(37,),dtype='int64')
    bert=TFAlbertModel.from_pretrained('albert-base-v2', return_dict=True)(input_layer)
    layer1=bert[1]
    classfier=tf.keras.layers.Dense(1,activation='sigmoid')(layer1)
    model=tf.keras.Model(inputs=input_layer,outputs=classfier)
    #model.summary()
    model.load_weights(filename)
    return model

In [5]:
model1=import_model('./checkpoints/bert_model6')
model2=import_model('./checkpoints/bert_model5')
model3=import_model('./checkpoints/bert_model4')
model4=import_model('./checkpoints/bert_model3')
model5=import_model('./checkpoints/bert_model2')

Some weights of the model checkpoint at albert-base-v2 were not used when initializing TFAlbertModel: ['predictions']
- This IS expected if you are initializing TFAlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFAlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFAlbertModel were initialized from the model checkpoint at albert-base-v2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFAlbertModel for predictions without further training.
Some weights of the model checkpoint at albert-base-v2 were not used when initializing TFAlbertModel: ['predictions']
- This IS expected if you are initializing TFAlbertM